<a href="https://colab.research.google.com/github/entangelk/study_AIs/blob/main/docs/quests/NLPs/classification_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install wget

In [ ]:
# import wget

In [ ]:
# wget.download('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')

In [ ]:
import pandas as pd

/tmp/ipykernel_1349/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
get_data = pd.read_csv('ratings_train.txt', sep='\t')
pd.read_csv('ratings_train.txt', sep='\t')[:3]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [ ]:
# !apt-get -y install fonts-nanum
# !fc-cache -fv
# !rm ~/.cache/matplotlib -rf

## 데이터 전처리

In [ ]:
get_data.isna().sum()

id          0
document    5
label       0
dtype: int64

In [ ]:
get_data=get_data.dropna()

In [ ]:
import re

clear_list = []
for i in get_data['document']:
    get_text = re.sub(r'\.', ' ', i)
    get_text = get_text.lower()
    clear_list.append(get_text)

- 마침표, 말줄임표는 부정 긍정에 모두 포함되어 있으므로 삭제
- 숫자는 연령대에 대한 내용, 연도에 대한 감상이 담길 수 있으므로 살림
- 영어는 소문자로 모두 처리하여 진행

In [ ]:
clear_list[:5]

['아 더빙   진짜 짜증나네요 목소리',
 '흠   포스터보고 초딩영화줄    오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼   솔직히 재미는 없다  평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다']

In [ ]:
# !pip install python-mecab-ko

In [ ]:
from mecab import MeCab
mecab = MeCab()

In [ ]:
mecab.pos(clear_list[5])

[('막', 'MAJ'),
 ('걸음마', 'NNG'),
 ('뗀', 'VV+ETM'),
 ('3', 'SN'),
 ('세', 'NNBC'),
 ('부터', 'JX'),
 ('초등', 'NNG'),
 ('학교', 'NNG'),
 ('1', 'SN'),
 ('학년', 'NNG'),
 ('생', 'XSN'),
 ('인', 'VCP+ETM'),
 ('8', 'SN'),
 ('살', 'NNBC'),
 ('용', 'XSN'),
 ('영화', 'NNG'),
 ('ㅋㅋㅋ', 'IC'),
 ('별반', 'MAG'),
 ('개', 'NNG'),
 ('도', 'JX'),
 ('아까움', 'VA+ETN')]

In [ ]:
def mecabing (get_list):
    # 1차 가공
    noun_sym_list=[]
    for i in get_list:
        morphas = mecab.pos(i)
        list_text = ''
        for k,v in morphas:
            if v == 'MAG' or v == 'NNG' or v =='SY' or v =='VA+ETN' or v=='SN':
                list_text += k
                list_text += ' '
            if k in r'[ㄱ-ㅣ]':
                list_text += k
                list_text += ' '
        noun_sym_list.append(list_text)

    # 카운팅을 위한 스플릿
    extend_text_list = []
    for i in noun_sym_list:
        temp_list = i.split()
        for j in temp_list:
            extend_text_list.append(j)
    # 카운팅
    from collections import Counter
    count_dic = Counter(extend_text_list)

    # 2차 가공
    re_clear_list =[]
    for i in noun_sym_list:
        temp_text = ''
        for k,v in mecab.pos(i):
            if len(k) != 1:
                if count_dic.get(k,0) >= 3:
                    temp_text += k
                    temp_text += ' '
        re_clear_list.append(temp_text)

    return re_clear_list

In [ ]:
def mecabing_onelow (get_list):
    # 1차 가공
    noun_sym_list=[]
    for i in get_list:
        morphas = mecab.pos(i)
        list_text = ''
        for k,v in morphas:
            if v == 'MAG' or v == 'NNG' or v =='SY' or v =='VA+ETN' or v=='SN':
                list_text += k
                list_text += ' '
            if k in r'[ㄱ-ㅣ]':
                list_text += k
                list_text += ' '
        noun_sym_list.append(list_text)
    return noun_sym_list

In [ ]:
noun_sym_list = mecabing(clear_list)
noun_sym_list[:3]

['진짜 짜증 목소리 ', '초딩 영화 오버 연기 ', '너무 ']

In [ ]:
get_data['doc_clear'] = noun_sym_list
get_data[:5]

,id,document,label,doc_clear
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,진짜 짜증 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,초딩 영화 오버 연기
2,10265843,너무재밓었다그래서보는것을추천한다,0,너무
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 솔직히 재미 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,익살 연기 영화 너무나


## 벡터화

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(tokenizer=None, ngram_range=(1,2))

In [ ]:
tfidfVectorizer.fit(get_data['doc_clear'])

TfidfVectorizer(ngram_range=(1, 2))

## 트레인 셋, 테스트셋 분리

In [ ]:
train_set = get_data['doc_clear']
train_target = get_data['label']

In [ ]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(train_set, train_target, test_size=0.3, random_state=1125)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((104996,), (44999,), (104996,), (44999,))

In [ ]:
features_train[:3]

96377                                                
140613      영화 각본 내용 짐작 모두 가능 전형 스토리 스릴러 내용 짐작 별로 다임 
140499    세상 멸망 멸망 교육 솔직히 철학 포장 모순 덩어리 쓰레기 스토리 허점 떡밥 
Name: doc_clear, dtype: object

In [ ]:
features_test[:3]

4298                  내용 없이 
74187                       
149880    순수 감동 아름다움 영화 오랜만 
Name: doc_clear, dtype: object

## 모델

In [ ]:
train_vector_set = tfidfVectorizer.transform(features_train)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model

DecisionTreeClassifier()

In [ ]:
model.fit(train_vector_set,target_train)

DecisionTreeClassifier()

## 평가

In [ ]:
sample_nag = "와 진짜 구려요. 이런 영화는 바보도 만들듯?"

In [ ]:
sample_nag_mecab = mecabing_onelow([sample_nag])

In [ ]:
sample_nag_mecab

['진짜 영화 바보 ']

In [ ]:
model.predict(tfidfVectorizer.transform(sample_nag_mecab))

array([0])

In [ ]:
sample_pos = "너무 감동적이에요. 진짜 재미있었어요!"

In [ ]:
sample_pos_mecab = mecabing_onelow([sample_pos])

In [ ]:
sample_pos_mecab

['너무 감동 진짜 ']

In [ ]:
model.predict(tfidfVectorizer.transform(sample_pos_mecab))

array([1])

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
target_predict = model.predict(tfidfVectorizer.transform(features_test))
print(classification_report(target_test, target_predict))

              precision    recall  f1-score   support

           0       0.70      0.63      0.66     22458
           1       0.67      0.73      0.70     22541

    accuracy                           0.68     44999
   macro avg       0.68      0.68      0.68     44999
weighted avg       0.68      0.68      0.68     44999



## 1차 모델 평가결과 (테스트 사이즈 0.3)

```
              precision    recall  f1-score   support

           0       0.70      0.63      0.66     22563
           1       0.66      0.73      0.69     22436

    accuracy                           0.68     44999
   macro avg       0.68      0.68      0.68     44999
weighted avg       0.68      0.68      0.68     44999
```

f1 score간 차이가 적고 가중평균이 높아 모델의 정확도에 대한 신뢰성은 높은 편이나 f1 socre와 정확도가 0.7을 하회하여 정확도에 대한 처리를 더욱더 하면 좋을 것 같다.

## 2차 모델 평가결과 (테스트 사이즈 0.1, 단어사전 재정리)

```
              precision    recall  f1-score   support

           0       0.71      0.64      0.67      7550
           1       0.67      0.73      0.70      7450

    accuracy                           0.69     15000
   macro avg       0.69      0.69      0.69     15000
weighted avg       0.69      0.69      0.69     15000
```

테스트 케이스를 조정하고 단어사전 재정리를 하니 조금? 올랐다

## 3차 모델 평가결과 (테스트 사이즈 0.3, 단어사전 재정리)

```
              precision    recall  f1-score   support

           0       0.70      0.63      0.66     22458
           1       0.67      0.72      0.69     22541

    accuracy                           0.68     44999
   macro avg       0.68      0.68      0.68     44999
weighted avg       0.68      0.68      0.68     44999
```

1(긍정)에 대한 값이 조금 조정되었으나, 유의미한 차이는 없었다

## 4차 모델 평가결과 (테스트 사이즈 0.3, 단어사전 재정리, ngram = (1,3))

```
              precision    recall  f1-score   support

           0       0.70      0.63      0.66     22458
           1       0.67      0.73      0.70     22541

    accuracy                           0.68     44999
   macro avg       0.68      0.68      0.68     44999
weighted avg       0.68      0.68      0.68     44999
```

ngram = (1,3) 은 유의미한 차이를 만들어내지 못했으나 1번그룹(긍정)에 대한 f1 score가 조금 상승했다.

## 5차 모델 평가결과 (테스트 사이즈 0.3, 단어사전 재정리, ngram = (1,2))

```
              precision    recall  f1-score   support

           0       0.70      0.63      0.66     22458
           1       0.67      0.73      0.70     22541

    accuracy                           0.68     44999
   macro avg       0.68      0.68      0.68     44999
weighted avg       0.68      0.68      0.68     44999
```

ngram = (1,2) 은 유의미한 차이를 만들어내지 못했다.